## Multipole covariance

In [ ]:
import sys
sys.path.insert(1, '/home/joeadamo/Research/SPHEREx/thecov')
import thecov.base as base
import thecov.math as math
import numpy as np
from thecov import SurveyGeometry, GaussianCovariance
import mockfactory
import cosmoprimo

In [ ]:
# Define random catalogs
random_file = "/home/joeadamo/Research/SPHEREx/thecov/data/LRG_ffa_NGC_0_clustering.ran.fits"
randoms = mockfactory.Catalog.read(random_file)

# Define fiducial cosmology used in calculations
cosmo = cosmoprimo.fiducial.DESI()

# Should define FKP weights column with this name
# randoms['WEIGHT_FKP'] = 1./(1. + 1e4*randoms['NZ'])  # FKP weights are optional

# Convert sky coordinates to cartesian using fiducial cosmology
randoms['POSITION'] = mockfactory.utils.sky_to_cartesian(
                          cosmo.comoving_radial_distance(randoms['Z']),
                          randoms['RA'],
                          randoms['DEC'],
                          degree=True)


In [ ]:
# define SurveyGeometry object
alpha = 0.1

nmesh=16
boxsize=4000
boxpad = 1.4
kmin = 0
kmax = 0.01
dk = 0.005
nthreads = 4
mask_ellmax = 6

geometry = SurveyGeometry(randoms, alpha, nmesh=nmesh, boxpad=boxpad, 
                          kmin=kmin, kmax=kmax, dk=dk, mask_ellmax=mask_ellmax,
                          nthreads=nthreads, sample_mode='monte-carlo')


In [ ]:
geometry.compute_window_kernels()

In [ ]:
import itertools as itt
W_ABCD = base.SparseNDArray.load("/home/joeadamo/Research/SPHEREx/thecov/thecov/cache/W_ABCD.npz")
pk_ellmax=4
delta_k_max = 3

G = geometry.get_gaunt_coefficients(mask_ellmax=mask_ellmax, pk_ellmax=pk_ellmax)

product = G @ W_ABCD
result = np.zeros((list(product.shape_in) + [3,3,3,3]), dtype=np.complex128)

WinKernel = np.zeros((2*delta_k_max+1, pk_ellmax//2+1, pk_ellmax//2+1, pk_ellmax//2+1, pk_ellmax//2+1), dtype=np.float64)

print(WinKernel.shape)
# print(G.shape_in, G.shape_out)
# print(W_ABCD.shape_in, W_ABCD.shape_out)
print(product.shape_in, product.shape_out)
print(f"result takes up {result.nbytes/1024/1024}MB")

# product = [4,4,4] [3,3,3,3,9,9,9,9]

# result = [7,3,3,3,3]

for l in range(0, pk_ellmax+1, 2):
    l_idx = int(l / 2)

    #print(l, l_idx)

# NOTE: This for-loop is slow!
for l1, l2, l3, l4 in itt.product(np.arange(0, pk_ellmax+1, 2), repeat=4):
    l1_idx = int(l1 / 2)
    l2_idx = int(l2 / 2)
    l3_idx = int(l3 / 2)
    l4_idx = int(l4 / 2)
    
    for m1, m2, m3, m4 in itt.product(*[np.arange(-l, l+1, 2) for l in (l1, l2, l3, l4)]):
        m1_idx = (m1 + l1) / 2
        m2_idx = (m2 + l2) / 2
        m3_idx = (m3 + l3) / 2
        m4_idx = (m4 + l4) / 2

        W_times_G = product[l1_idx,l2_idx,l3_idx,l4_idx,m1_idx,m2_idx,m3_idx,m4_idx]
        #print(test.shape)
        W_times_G *= 2.12
        result[:,:,:,l1_idx,l2_idx,l3_idx,l4_idx] = W_times_G.toarray().reshape(product.shape_in)
        #result[l1_idx,l2_idx,l3_idx,l4_idx] += product[l1_idx,l2_idx,l3_idx,l4_idx,m1_idx,m2_idx,m3_idx,m4_idx]# * \
                                # math.get_real_Ylm(l1, m1)(k1xh, k1yh, k1zh) * \
                                # math.get_real_Ylm(l2, m2)(k2xh, k2yh, k2zh) * \
                                # math.get_real_Ylm(l3, m3)(k1xh, k1yh, k1zh) * \
                                # math.get_real_Ylm(l4, m4)(k2xh, k2yh, k2zh)
        assert 1 == 0

# for delta_k in range(-delta_k_max, delta_k_max + 1):
#         modes = (k2_bin_index - k1_bin_index == delta_k)
#         WinKernel[delta_k] = result[modes]

In [ ]:
kmin = 0
kmax = 0.05
dk = 0.005
boxsize=8000
kmodes_sampled = 1000
kmodes, Nmodes, weights = math.sample_kmodes(kmin=kmin,
                                                kmax=kmax,
                                                dk=dk,
                                                boxsize=boxsize,
                                                max_modes=kmodes_sampled,
                                                k_shell_approx=0.1,
                                                sample_mode="monte-carlo")

print(kmodes[10].shape, weights[10])
print(len(kmodes), len(weights))

## Old test code

In [ ]:
matrix_a = np.array([[1, 2], [3, 4]])
matrix_b = np.array([[5, 6, 7], [8, 9, 10], [11,12,13]])
matrix_c = np.array([[9, 10], [11, 12], [13,14]])
matrix_d = np.array([[13, 14, 15], [16, 17, 18]])


In [ ]:
cov = thecov.base.MultipoleCovariance(symmetric=False)

In [ ]:
cov.set_ell_cov(0,0,thecov.base.Covariance(matrix_a))
cov.set_ell_cov(2,2,thecov.base.Covariance(matrix_b))
cov.set_ell_cov(0,2,thecov.base.Covariance(matrix_d))
cov.set_ell_cov(2,0,thecov.base.Covariance(matrix_c))

In [ ]:
cov.cov

In [ ]:
cov.cov = cov.cov

In [ ]:
cov.cov

In [ ]:
cov._multipole_covariance

In [ ]:
cov.ells = (0,2), (0,2,4)

In [ ]:
cov._multipole_covariance

## Gaunt coefficients

In [ ]:
import scipy
import sympy.physics.wigner
import numpy as np
import itertools as itt
import sys
sys.path.insert(1, '/home/oalves/thecov')
import thecov.base

In [ ]:
# Define the shape of the multi-dimensional sparse array
shape_out = (3, 3, 3, 3, 5, 5, 5, 5) # l1, l2, l3, l4, m1, m2, m3, m4
shape_in = (7, 7, 13, 13) # la, lb, ma, mb

In [ ]:
coefficients = thecov.base.SparseNDArray(shape_out, shape_in)

for l1, l2, l3, l4 in itt.product((0,2,4), repeat=4):
    for m1, m2, m3, m4 in itt.product(*[np.arange(-l, l+1, 2) for l in (l1, l2, l3, l4)]):
        for la in np.arange(np.abs(l1-l4), l1+l4+1, 2):
            for lb in np.arange(np.abs(l2-l3), l2+l3+1, 2):
                for ma, mb in itt.product(*[np.arange(-l, l+1, 2) for l in (la, lb)]):
                    value = np.float64(sympy.physics.wigner.gaunt(l1,l4,la,m1,m4,ma)*\
                                       sympy.physics.wigner.gaunt(l2,l3,lb,m2,m3,mb))
                    if value != 0.:
                        coefficients[l1//2,l2//2,l3//2,l4//2,
                                     m1//2+2,m2//2+2,m3//2+2,m4//2+2,
                                     la//2,lb//2,
                                     ma//2+6,mb//2+6] += value
                        
        for lc in np.arange(np.abs(l1-l2), l1+l2+1, 2):
            for la in np.arange(np.abs(lc-l4), lc+l4+1, 2):
                for ma, mc in itt.product(*[np.arange(-l, l+1, 2) for l in (la, lc)]):
                    value = np.float64(sympy.physics.wigner.gaunt(l1,l2,lc,m1,m2,mc)*\
                                       sympy.physics.wigner.gaunt(lc,l4,la,mc,m4,ma))
                    lb, mb = l3, m3
                    if value != 0.:
                        coefficients[l1//2,l2//2,l3//2,l4//2,
                                     m1//2+2,m2//2+2,m3//2+2,m4//2+2,
                                     la//2,lb//2,
                                     ma//2+6,mb//2+6] += value
coefficients

In [ ]:
pk[0:300:5]

In [ ]:
60125/(50625*8281)

In [ ]:
coefficients.save('cosmic_variance_coefficients.npz')

In [ ]:
coefficients = thecov.base.SparseNDArray.load('cosmic_variance_coefficients.npz')
coefficients

In [ ]:
(coefficients @ np.ones((*shape_in, 3))).shape